In [130]:
from keras.models import Model, load_model
import numpy as np
import tensorflow as tf
import math
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import time
from keras import Model
from keras.layers import Input, Dense, GRU, Embedding
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [131]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/bin/python3
3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)


In [132]:
lines = open('cornell/movie_lines128.txt').read().split('\n')
len(lines)



304714

In [133]:
conv_lines = open('cornell/movie_conversations128.txt').read().split('\n')
conv_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [134]:
id2line = {}
for line in lines:
    line_list = line.split(' +++$+++ ')
    if len(line_list) == 5:
        id2line[line_list[0]] = line_list[4]
    else :
        print(line_list)
        
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ","")
    convs.append(_line.split(','))

print(convs[:10])


['']
[['L194', 'L195', 'L196', 'L197'], ['L198', 'L199'], ['L200', 'L201', 'L202', 'L203'], ['L204', 'L205', 'L206'], ['L207', 'L208'], ['L271', 'L272', 'L273', 'L274', 'L275'], ['L276', 'L277'], ['L280', 'L281'], ['L363', 'L364'], ['L365', 'L366']]


In [135]:
mark_start = 'ssss '
mark_end = ' eeee'
input_x = []
output_y = []
for conv in convs:
    for i in range(len(conv) - 1):
        input_x.append(id2line[conv[i]])
        output_y.append(mark_start+id2line[conv[i+1]]+mark_end)

In [138]:
for i in range(1,10):
    print(input_x[i])
    print(output_y[i])

Well, I thought we'd start with pronunciation, if that's okay with you.
ssss Not the hacking and gagging and spitting part.  Please. eeee
Not the hacking and gagging and spitting part.  Please.
ssss Okay... then how 'bout we try out some French cuisine.  Saturday?  Night? eeee
You're asking me out.  That's so cute. What's your name again?
ssss Forget it. eeee
No, no, it's my fault -- we didn't have a proper introduction ---
ssss Cameron. eeee
Cameron.
ssss The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does. eeee
The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.
ssss Seems like she could get a date easy enough... eeee
Why?
ssss Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something. eeee
Unsolved mystery.  She used to be really popular when she started high schoo

In [136]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "th-at is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [137]:
input_x_clean = []
for sent in input_x:
    input_x_clean.append(clean_text(sent))

output_y_clean = []
for sent in output_y:
    output_y_clean.append(clean_text(sent))
    
for i in range(1,10):
    print(input_x_clean[i])
    print(output_y_clean[i])
    
min_length = 2
max_length = 20
final_input = []
final_output = []
for i in range(0,len(input_x_clean)):
    temp_input_list = input_x_clean[i].split(' ')
    temp_output_list = output_y_clean[i].split(' ')
    if len(temp_input_list) > max_length or len(temp_input_list) < min_length  or len(temp_output_list) > max_length or len(temp_output_list) < min_length:
        d  = 1 + 1
    else:
        final_input.append(input_x_clean[i])
        final_output.append(output_y_clean[i])



well i thought we would start with pronunciation if that is okay with you
ssss not the hacking and gagging and spitting part  please eeee
not the hacking and gagging and spitting part  please
ssss okay then how about we try out some french cuisine  saturday  night eeee
you are asking me out  that is so cute that is your name again
ssss forget it eeee
no no it is my fault  we did not have a proper introduction 
ssss cameron eeee
cameron
ssss the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does eeee
the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does
ssss seems like she could get a date easy enough eeee
why
ssss unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something eeee
unsolved mystery  she used to be really popular when she started high school then it was just like she got 

In [139]:
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Reverse the tokens.
            tokens = np.flip(tokens, axis=1)

            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        if padding:
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

In [140]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "th-at is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [141]:
num_words = 10000
tokenizer_src = TokenizerWrap(texts = final_input, padding='pre', reverse = True, num_words = num_words)

In [142]:
tokenizer_dest = TokenizerWrap(texts = final_output, padding='post', reverse = False, num_words = num_words)
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)
token_start = tokenizer_dest.word_index[mark_start.strip()]
print(token_start)
token_end = tokenizer_dest.word_index[mark_end.strip()]
print(token_end)
print(tokens_dest[2])
print(tokens_src[2])
print (tokens_dest[5])
print(tokens_src[5])
encoder_input_data = tokens_src[:,:]
decoder_input_data = tokens_dest[:,:-1]
decoder_output_data = tokens_dest[:,1:]
print(decoder_input_data.shape)
print(decoder_output_data.shape)


(144852, 16)
(144852, 17)
2
1
[  2 275   8   1   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0 156 144  22   4   9 920  42   4   9  46  14 500  11   1]
[   2   50   69    3   13  352   26    5 7793    1    0    0    0    0
    0    0    0]
[  0   0   0   0   0   0   0   0   0   0   0 370  27   4  21 822]
(144852, 16)
(144852, 16)


In [143]:
def sparse_cross_entropy(y_true, y_pred):
    """
    Calculate the cross-entropy loss between y_true and y_pred.
    
    y_true is a 2-rank tensor with the desired output.
    The shape is [batch_size, sequence_length] and it
    contains sequences of integer-tokens.

    y_pred is the decoder's output which is a 3-rank tensor
    with shape [batch_size, sequence_length, num_words]
    so that for each sequence in the batch there is a one-hot
    encoded array of length num_words.
    """

    # Calculate the loss. This outputs a
    # 2-rank tensor of shape [batch_size, sequence_length]
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true,
                                                          logits=y_pred)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire 2-rank tensor, we reduce it
    # to a single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

In [144]:
import keras.losses
keras.losses.sparse_cross_entropy = sparse_cross_entropy

In [145]:
model = load_model('chbot3.h5')

In [146]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 128)    1280000     encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_gru1 (GRU)              (None, None, 512)    984576      encoder_embedding[0][0]          
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
encoder_gr

In [147]:
print(model.input_names)
print(model.output_names)
print(model.layers)
print (model.layers[0])
print (model.layers[1])
print (model.inputs[1])

['encoder_input', 'decoder_input']
['decoder_output']
[<keras.engine.input_layer.InputLayer object at 0x7fcad097def0>, <keras.layers.embeddings.Embedding object at 0x7fcad091c5c0>, <keras.layers.recurrent.GRU object at 0x7fcad06fec18>, <keras.engine.input_layer.InputLayer object at 0x7fcad097f7f0>, <keras.layers.recurrent.GRU object at 0x7fcad0662550>, <keras.layers.embeddings.Embedding object at 0x7fcacd4656d8>, <keras.layers.recurrent.GRU object at 0x7fcacd465b70>, <keras.layers.recurrent.GRU object at 0x7fcacd465a90>, <keras.layers.recurrent.GRU object at 0x7fcacd4650f0>, <keras.layers.recurrent.GRU object at 0x7fcacd465208>, <keras.layers.core.Dense object at 0x7fcacd465438>]
Tensor("decoder_input_14:0", shape=(?, ?), dtype=float32)


In [148]:
encoder_input = model.input[0]
encoder_output = model.layers[6].output
model_encoder = Model(input=[encoder_input], outputs = [encoder_output])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [149]:
model_encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, None)              0         
_________________________________________________________________
encoder_embedding (Embedding (None, None, 128)         1280000   
_________________________________________________________________
encoder_gru1 (GRU)           (None, None, 512)         984576    
_________________________________________________________________
encoder_gru2 (GRU)           (None, None, 512)         1574400   
_________________________________________________________________
encoder_gru3 (GRU)           (None, 512)               1574400   
Total params: 5,413,376
Trainable params: 5,413,376
Non-trainable params: 0
_________________________________________________________________


In [150]:
state_size = 512
decoder_input = model.inputs[1]
decoder_initial_state = Input(shape=(state_size,),name = 'decoder_initial_state')
decoder_embedding = model.layers[5]
decoder_gru1 = model.layers[7]
decoder_gru2 = model.layers[8]
decoder_gru3 = model.layers[9]
decoder_dense = model.layers[10]


In [151]:
net = decoder_input
net = decoder_embedding(net)
net = decoder_gru1(net, initial_state=decoder_initial_state)

In [152]:
net = decoder_gru2(net, initial_state=decoder_initial_state)

In [153]:
net = decoder_gru3(net, initial_state=decoder_initial_state)


In [158]:
decoder_output = decoder_dense(net)

In [159]:
model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs = [decoder_output])

In [160]:
model_decoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, None, 128)    1280000     decoder_input[0][0]              
__________________________________________________________________________________________________
decoder_initial_state (InputLay (None, 512)          0                                            
__________________________________________________________________________________________________
decoder_gru1 (GRU)              (None, None, 512)    984576      decoder_embedding[1][0]          
                                                                 decoder_initial_state[0][0]      
__________

In [161]:
def translate(input_text, true_output_text=None):
    """Translate a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding=True)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)

    # Max number of tokens / words in the output sequence.
    max_tokens = tokenizer_dest.max_tokens

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict(), but it would make the code
        # much more complicated.

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()

    # Print the translated output-text.
    print("Translated text:")
    print(output_text)
    print()

    # Optionally print the true translated text.
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

In [162]:
translate(input_text="how are you.")
translate(input_text="what are you doing")

Input text:
how are you.

Translated text:
 fine fine eeee

Input text:
what are you doing

Translated text:
 i am going in i have to think you were just movie eeee



In [163]:
translate(input_text = "Do you love me")

Input text:
Do you love me

Translated text:
 i asked you too eeee



In [164]:
translate(input_text = "what is your name")
translate(input_text = "how are you")

Input text:
what is your name

Translated text:
 my name is sir sir of eeee

Input text:
how are you

Translated text:
 fine fine eeee



In [165]:
translate(input_text = "Do you like me")

Input text:
Do you like me

Translated text:
 yes i like you eeee



In [167]:
translate(input_text = "where do you live")

Input text:
where do you live

Translated text:
 excuse me eeee



In [168]:
translate(input_text = "lets go out today")

Input text:
lets go out today

Translated text:
 uhhuh how many i would not do it eeee



In [170]:
translate(input_text = "can i visit you today")

Input text:
can i visit you today

Translated text:
 yeah sure eeee



In [171]:
translate(input_text="what will you eat today")

Input text:
what will you eat today

Translated text:
 a coke eeee

